In [14]:
import json
import codecs
from distance import levenshteinDistance

# Getting initial list of bad words

English bad words translated to bulgarian. <br>
And from bulgarian to english. <br>

In [5]:
with open("data/english-bad-words.txt") as file:
    english_original = [line.rstrip() for line in file]

with open("data/english-bad-words-translated.txt") as file:
    english_translated = [line.rstrip() for line in file]

with open("data/english-bad-words-translated-back.txt") as file:
    english_translated_back = [line.rstrip() for line in file]

We will filter the words that does not translate back to the original word.

In [28]:
zipped = zip(english_original, english_translated, english_translated_back)
stable_translations = set([
    translation[1] for translation in zipped
        if levenshteinDistance(translation[0], translation[2]) < 3
        and levenshteinDistance(translation[0], translation[1]) > 3
    ])

In [32]:
json_object = json.dumps(list(stable_translations), indent=4, ensure_ascii=False)
with codecs.open("data/bad_words_translated.json", "w", "utf-8") as outfile:
    outfile.write(json_object)

# Classifying bad words from BG Jargon

In [10]:
with open('data/bgjargon.json', 'r', encoding="utf-8") as f:
    dictionary = json.load(f)

with open('data/bgjargon_words.json', 'r', encoding="utf-8") as f:
    words = set(json.load(f))

with open('data/bad_words_translated.json', 'r', encoding="utf-8") as f:
    translated_words = set(json.load(f))

Get all words from bg jargon that have tags - some of the translated bad words

In [32]:
dictionary_bad_tags = {}

for key, value in dictionary.items():
    if key in translated_words or any(map(lambda x: x in translated_words, value['tags'])):
        dictionary_bad_tags[key] = value

len(dictionary_bad_tags)

2563

Second iteration - all words that have tags which are already marked as bad

In [33]:
dictionary_bad_tags_deep = dictionary_bad_tags.copy()

for key, value in dictionary.items():
    if any(map(lambda x: x in dictionary_bad_tags.keys(), value['tags'])):
        dictionary_bad_tags_deep[key] = value

len(dictionary_bad_tags_deep)

3653

In [34]:
json_object = json.dumps(dictionary_bad_tags_deep, indent=4, ensure_ascii=False)
with codecs.open("data/bgjargon_bad_tags.json", "w", "utf-8") as outfile:
    outfile.write(json_object)

Same operation as before but equals is replaced with levenshtein distance < 2

In [36]:
dictionary_bad_tags_ld = {}

for key, value in dictionary.items():
    for word in translated_words:
        if levenshteinDistance(word, key) < 2 or any(map(lambda x: levenshteinDistance(x, word) < 2, value['tags'])):
            dictionary_bad_tags_ld[key] = value
            break

len(dictionary_bad_tags_ld)

3108

In [37]:
dictionary_bad_tags_deep_ld = dictionary_bad_tags_ld.copy()

for key, value in dictionary.items():
    if any(map(lambda x: x in dictionary_bad_tags_ld.keys(), value['tags'])):
        dictionary_bad_tags_deep_ld[key] = value

len(dictionary_bad_tags_deep_ld)

4261

In [38]:
json_object = json.dumps(dictionary_bad_tags_deep_ld, indent=4, ensure_ascii=False)
with codecs.open("data/bgjargon_bad_tags_ld.json", "w", "utf-8") as outfile:
    outfile.write(json_object)